#Numba

1. https://colab.research.google.com/github/cbernet/maldives/blob/master/numba/numba_cuda.ipynb#scrollTo=Ctr6aM3cmkdx

2. https://stackoverflow.com/questions/48811248/how-to-use-numba-in-colaboratory

In [1]:
!find / -iname 'libdevice'

/usr/local/lib/python3.7/dist-packages/jaxlib/cuda/nvvm/libdevice
/usr/local/cuda-11.2/nvvm-prev/libdevice
/usr/local/cuda-11.2/nvvm/libdevice


In [2]:
!find / -iname 'libnvvm.so'

find: ‘/proc/45/task/45/net’: Invalid argument
find: ‘/proc/45/net’: Invalid argument
/usr/local/cuda-11.2/nvvm-prev/lib64/libnvvm.so
/usr/local/cuda-11.2/nvvm/lib64/libnvvm.so


In [1]:
import os
os.environ['NUMBAPRO_LIBDEVICE'] = "/usr/local/cuda-11.2/nvvm/libdevice"
os.environ['NUMBAPRO_NVVM'] = "/usr/local/cuda-11.2/nvvm-prev/lib64/libnvvm.so"

In [2]:
from numba import jit

#Package Section

In [4]:
import sys
import numpy as np
import copy
from numpy import linalg as LA
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.cluster import KMeans
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn import metrics
import time
# for sparse matrix
from scipy import sparse
#early stop
from keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint


#Classes and functions with Numba Decorator

In [6]:
# invalide devide resutls will be nan
np.seterr(divide='ignore', invalid='ignore')

############------------graph_encoder_embed_start----------------###############

def GEE_run(Y, n, X=np.array([[]]), **kwargs):
    defaultKwargs = {'DiagA': True, 'Laplacian': False, 'Correlation': True}
    kwargs = { **defaultKwargs, **kwargs}
    
    X = to_s3_list(X)
    
    emb_strat = time.time()

    if kwargs['DiagA']:
      X = Diagonal(X, n)

    if kwargs['Laplacian']:
      X = Laplacian(X, n)
    
    Z, W = GEE_Basic(X, Y, n)

    if kwargs['Correlation']:
      Z = Correlation(Z)
    
    emb_end = time.time()
    emb_time = emb_end - emb_strat
    
    return Z, W, emb_time

@jit(nopython = True)
def GEE_Basic(X, Y, n):
  """
    graph embedding basic function
    input X is S3 edge list
    input Y is numpy array with size (n,1):
    -- value -1 indicate no lable
    -- value >=0 indicate real label
    input n: number of vertices
  """
  k = Y[:,0].max() + 1

  Z = np.zeros((n,k))   
  #nk: 1*n array, contains the number of observations in each class
  #W: encoder marix. W[i,k] = {1/nk if Yi==k, otherwise 0}
  nk = np.zeros((1,k))
  W = np.zeros((n,k))

  for i in range(k):
    nk[0,i] = np.count_nonzero(Y[:,0]==i)

  for i in range(Y.shape[0]):
    k_i = Y[i,0]
    if k_i >=0:
      W[i,k_i] = 1/nk[0,k_i] ## GEE paper
      # W[i,k_i] = nk[0,k_i]/n ## use as an example to show if people want to use nk/n instead of 1/nk

  for row in X:
    [v_i, v_j, edg_i_j] = row
    v_i = int(v_i)
    v_j = int(v_j)

    label_i = Y[v_i][0] 
    label_j = Y[v_j][0]

    if label_j >= 0:
      Z[v_i, label_j] = Z[v_i, label_j] + W[v_j, label_j]*edg_i_j
    if (label_i >= 0) and (v_i != v_j):
      Z[v_j, label_i] = Z[v_j, label_i] + W[v_i, label_i]*edg_i_j

  return Z, W    
  

def Diagonal(X, n):
  # add self-loop to edg list -- add 1 connection for each (i,i)
  self_loops = np.column_stack((np.arange(n), np.arange(n), np.ones(n)))
  # faster than vstack --  adding the second to the bottom
  X = np.concatenate((X,self_loops), axis = 0)
  return X

def Laplacian(X, n):
  s = X.shape[0] # get the row number of the edg list

  D = np.zeros((n,1))
  for row in X:
    [v_i, v_j, edg_i_j] = row
    v_i = int(v_i)
    v_j = int(v_j)
    D[v_i] = D[v_i] + edg_i_j
    if v_i != v_j:
      D[v_j] = D[v_j] + edg_i_j

  D = np.power(D, -0.5)
  
  for i in range(s):
    X[i,2] = X[i,2] * D[int(X[i,0])] * D[int(X[i,1])]

  return X

def Correlation(Z):
  """
    Calculate each row's 2-norm (Euclidean distance). 
    e.g.row_x: [ele_i,ele_j,ele_k]. norm2 = sqr(sum(ele_i^2+ele_i^2+ele_i^2))
    then divide each element by their row norm
    e.g. [ele_i/norm2,ele_j/norm2,ele_k/norm2]
  """
  row_norm = LA.norm(Z, axis = 1)
  reshape_row_norm = np.reshape(row_norm, (n,1))
  Z = np.nan_to_num(Z/reshape_row_norm)

  return Z


def to_s3_list(X):
  """
    if input X only has 2 columns, make it into s3 edge list.
    this function will return a s3 edge list
    [node_i, node_j, weight]...
  """
  s = X.shape[0] # get the row number of the edg list
  if X.shape[1] == 2:
    # enlarge the edgelist to s*3 by adding 1 to the thrid position as adj(i,j)
    X = np.insert(X, 1, np.ones(s,1))

  return X

def adj_to_edg(A):
  """
    input is the symmetric adjacency matrix: A
    other variables in this function:
    s: number of edges
    return edg_list -- matrix format with shape(edg_sum,3):
    example row in edg_list(matrix): [vertex1, vertex2, connection weight from Adj matrix]
  """
  # check the len of the second dimenson of A
  if A.shape[1] <= 3:
    edg = A
  else:
    n = A.shape[0]
    # construct the initial edgg_list matrix with the size of (edg_sum, 3)
    edg_list = []
    for i in range(n):
      for j in range(i, n):
        if A[i,j] > 0:
          row = [i, j, A[i,j]]
          edg_list.append(row)
    edg = np.array(edg_list)
  return edg

############------------graph_encoder_embed_end------------------###############
############------------Sparse_supervised_learning_start---------###############

# https://www.kaggle.com/c/talkingdata-mobile-user-demographics/discussion/22567
# https://github.com/tkipf/pygcn/blob/1600b5b748b3976413d1e307540ccc62605b4d6d/pygcn/utils.py#L73

def batch_generator(X, y, k, batch_size, shuffle):
    number_of_batches = int(X.shape[0]/batch_size)
    counter = 0
    sample_index = np.arange(X.shape[0])
    if shuffle:
        np.random.shuffle(sample_index)
    while True:
        batch_index = sample_index[batch_size*counter:batch_size*(counter+1)]
        X_batch = X[batch_index,:]
        y_batch = to_categorical(y[batch_index], num_classes=k)
        counter += 1
        yield X_batch, y_batch
        if (counter == number_of_batches):
            if shuffle:
                np.random.shuffle(sample_index)
            counter = 0

class Hyperperameters:
  """
    define perameters for GNN.
    default values are for GNN learning -- "Leaner" ==2:
      embedding via partial label, then learn unknown label via two-layer NN

  """
  def __init__(self):
    # there is no scaled conjugate gradiant in keras optimiser, use defualt instead
    # use whatever default
    self.learning_rate = 0.01  # Initial learning rate.
    self.epochs = 100 #Number of epochs to train.
    self.hidden = 20 #Number of units in hidden layer 
    self.val_split = 0.1 #Split 10% of training data for validation
    self.loss = 'categorical_crossentropy' # loss function

class GNN:
  def __init__(self, DataSets):
    GNN.DataSets = DataSets
    GNN.hyperM = Hyperperameters()
    GNN.model = self.GNN_model()  #model summary: GNN.model.summary()
      
 
  def GNN_model(self):
    """
      build GNN model
    """
    hyperM = self.hyperM
    DataSets = self.DataSets

    z_train = DataSets.z_train
    k = DataSets.d

    feature_num = z_train.shape[1]
    
    model = keras.Sequential([
    keras.layers.Flatten(input_shape = (feature_num,)),  # input layer 
    keras.layers.Dense(hyperM.hidden, activation='relu'),  # hidden layer -- no tansig activation function in Keras, use relu instead
    keras.layers.Dense(k, activation='softmax') # output layer, matlab used softmax for patternnet default ??? max(opts.neuron,K)? opts 
    ])

    optimizer = keras.optimizers.Adam(learning_rate = hyperM.learning_rate)

    model.compile(optimizer='adam',
                  loss=hyperM.loss,
                  metrics=['accuracy'])

    return model
    
  def GNN_run(self, flag):
    """
      Train and test directly.
      Do not learn from the unknown labels.
    """
    gnn = copy.deepcopy(self)
    hyperM = gnn.hyperM
    DataSets = self.DataSets
    k = DataSets.d
    z_train = DataSets.z_train
    y_train = DataSets.y_train
    y_test = DataSets.y_test
    z_test = DataSets.z_test
    model = gnn.model    


    if flag == "direct":
      y_train_one_hot = to_categorical(y_train, num_classes=k)
      train_strat = time.time() 
      history = model.fit(z_train, y_train_one_hot, 
        validation_split=hyperM.val_split,
        epochs=hyperM.epochs, 
        shuffle=True,
        verbose=0)
    else:
      early_stopping_callback = EarlyStopping(monitor='loss', patience=5, verbose=0)
      checkpoint_callback = ModelCheckpoint('GNN.h5', monitor='loss', save_best_only=True, mode='min', verbose=0)
      
      train_strat = time.time()
      history = model.fit(batch_generator(z_train, y_train, k, 32, True),
                      epochs=hyperM.epochs,
                      steps_per_epoch=z_train.shape[0],
                      callbacks=[early_stopping_callback, checkpoint_callback],
                      verbose=0)
    train_end = time.time()
    train_time = train_end - train_strat 

    y_test_one_hot = to_categorical(y_test, num_classes=k) 
    # set verbose to 0 to silent the output
    test_loss, test_acc = gnn.model.evaluate(z_test,  y_test_one_hot, verbose=0) 
    return test_acc, train_time
############------------Sparse_supervised_learning_end---------###############


#Classes and functions(original)

In [5]:
# invalide devide resutls will be nan
np.seterr(divide='ignore', invalid='ignore')

############------------graph_encoder_embed_start----------------###############
class GraphEncoderEmbed:
  def run(self, X, Y, n, **kwargs):
    defaultKwargs = {'DiagA': True, 'Laplacian': False, 'Correlation': True}
    kwargs = { **defaultKwargs, **kwargs}
    
    X = self.to_s3_list(X)
    
    emb_strat = time.time()

    if kwargs['DiagA']:
      X = self.Diagonal(X, n)

    if kwargs['Laplacian']:
      X = self.Laplacian(X, n)
    
    Z, W = self.Basic(X, Y, n)

    if kwargs['Correlation']:
      Z = self.Correlation(Z)
    
    emb_end = time.time()
    emb_time = emb_end - emb_strat
    
    return Z, W, emb_time

  def Basic(self, X, Y, n):
    """
      graph embedding basic function
      input X is S3 edge list
      input Y is numpy array with size (n,1):
      -- value -1 indicate no lable
      -- value >=0 indicate real label
      input n: number of vertices
    """
    k = Y[:,0].max() + 1

    Z = np.zeros((n,k))   
    #nk: 1*n array, contains the number of observations in each class
    #W: encoder marix. W[i,k] = {1/nk if Yi==k, otherwise 0}
    nk = np.zeros((1,k))
    W = np.zeros((n,k))

    for i in range(k):
      nk[0,i] = np.count_nonzero(Y[:,0]==i)

    for i in range(Y.shape[0]):
      k_i = Y[i,0]
      if k_i >=0:
        W[i,k_i] = 1/nk[0,k_i] ## GEE paper
        # W[i,k_i] = nk[0,k_i]/n ## use as an example to show if people want to use nk/n instead of 1/nk
 
    for row in X:
      [v_i, v_j, edg_i_j] = row
      v_i = int(v_i)
      v_j = int(v_j)

      label_i = Y[v_i][0] 
      label_j = Y[v_j][0]

      if label_j >= 0:
        Z[v_i, label_j] = Z[v_i, label_j] + W[v_j, label_j]*edg_i_j
      if (label_i >= 0) and (v_i != v_j):
        Z[v_j, label_i] = Z[v_j, label_i] + W[v_i, label_i]*edg_i_j

    return Z, W

  def Diagonal(self, X, n):
    # add self-loop to edg list -- add 1 connection for each (i,i)
    self_loops = np.column_stack((np.arange(n), np.arange(n), np.ones(n)))
    # faster than vstack --  adding the second to the bottom
    X = np.concatenate((X,self_loops), axis = 0)
    return X

  def Laplacian(self, X, n):
    s = X.shape[0] # get the row number of the edg list

    D = np.zeros((n,1))
    for row in X:
      [v_i, v_j, edg_i_j] = row
      v_i = int(v_i)
      v_j = int(v_j)
      D[v_i] = D[v_i] + edg_i_j
      if v_i != v_j:
        D[v_j] = D[v_j] + edg_i_j

    D = np.power(D, -0.5)
    
    for i in range(s):
      X[i,2] = X[i,2] * D[int(X[i,0])] * D[int(X[i,1])]

    return X
  
  def Correlation(self, Z):
    """
      Calculate each row's 2-norm (Euclidean distance). 
      e.g.row_x: [ele_i,ele_j,ele_k]. norm2 = sqr(sum(ele_i^2+ele_i^2+ele_i^2))
      then divide each element by their row norm
      e.g. [ele_i/norm2,ele_j/norm2,ele_k/norm2]
    """
    row_norm = LA.norm(Z, axis = 1)
    reshape_row_norm = np.reshape(row_norm, (n,1))
    Z = np.nan_to_num(Z/reshape_row_norm)

    return Z
    
  def to_s3_list(self,X):
    """
      if input X only has 2 columns, make it into s3 edge list.
      this function will return a s3 edge list
      [node_i, node_j, weight]...
    """
    s = X.shape[0] # get the row number of the edg list
    if X.shape[1] == 2:
      # enlarge the edgelist to s*3 by adding 1 to the thrid position as adj(i,j)
      X = np.insert(X, 1, np.ones(s,1))

    return X

############------------graph_encoder_embed_end------------------###############
############------------Sparse_supervised_learning_start---------###############

# https://www.kaggle.com/c/talkingdata-mobile-user-demographics/discussion/22567
# https://github.com/tkipf/pygcn/blob/1600b5b748b3976413d1e307540ccc62605b4d6d/pygcn/utils.py#L73

def batch_generator(X, y, k, batch_size, shuffle):
    number_of_batches = int(X.shape[0]/batch_size)
    counter = 0
    sample_index = np.arange(X.shape[0])
    if shuffle:
        np.random.shuffle(sample_index)
    while True:
        batch_index = sample_index[batch_size*counter:batch_size*(counter+1)]
        X_batch = X[batch_index,:]
        y_batch = to_categorical(y[batch_index], num_classes=k)
        counter += 1
        yield X_batch, y_batch
        if (counter == number_of_batches):
            if shuffle:
                np.random.shuffle(sample_index)
            counter = 0

class Hyperperameters:
  """
    define perameters for GNN.
    default values are for GNN learning -- "Leaner" ==2:
      embedding via partial label, then learn unknown label via two-layer NN

  """
  def __init__(self):
    # there is no scaled conjugate gradiant in keras optimiser, use defualt instead
    # use whatever default
    self.learning_rate = 0.01  # Initial learning rate.
    self.epochs = 100 #Number of epochs to train.
    self.hidden = 20 #Number of units in hidden layer 
    self.val_split = 0.1 #Split 10% of training data for validation
    self.loss = 'categorical_crossentropy' # loss function

class GNN:
  def __init__(self, DataSets):
    GNN.DataSets = DataSets
    GNN.hyperM = Hyperperameters()
    GNN.model = self.GNN_model()  #model summary: GNN.model.summary()
      
 
  def GNN_model(self):
    """
      build GNN model
    """
    hyperM = self.hyperM
    DataSets = self.DataSets

    z_train = DataSets.z_train
    k = DataSets.d

    feature_num = z_train.shape[1]
    
    model = keras.Sequential([
    keras.layers.Flatten(input_shape = (feature_num,)),  # input layer 
    keras.layers.Dense(hyperM.hidden, activation='relu'),  # hidden layer -- no tansig activation function in Keras, use relu instead
    keras.layers.Dense(k, activation='softmax') # output layer, matlab used softmax for patternnet default ??? max(opts.neuron,K)? opts 
    ])

    optimizer = keras.optimizers.Adam(learning_rate = hyperM.learning_rate)

    model.compile(optimizer='adam',
                  loss=hyperM.loss,
                  metrics=['accuracy'])

    return model
    
  def GNN_run(self, flag):
    """
      Train and test directly.
      Do not learn from the unknown labels.
    """
    gnn = copy.deepcopy(self)
    hyperM = gnn.hyperM
    DataSets = self.DataSets
    k = DataSets.d
    z_train = DataSets.z_train
    y_train = DataSets.y_train
    y_test = DataSets.y_test
    z_test = DataSets.z_test
    model = gnn.model    


    if flag == "direct":
      y_train_one_hot = to_categorical(y_train, num_classes=k)
      train_strat = time.time() 
      history = model.fit(z_train, y_train_one_hot, 
        validation_split=hyperM.val_split,
        epochs=hyperM.epochs, 
        shuffle=True,
        verbose=0)
    else:
      early_stopping_callback = EarlyStopping(monitor='loss', patience=5, verbose=0)
      checkpoint_callback = ModelCheckpoint('GNN.h5', monitor='loss', save_best_only=True, mode='min', verbose=0)
      
      train_strat = time.time()
      history = model.fit(batch_generator(z_train, y_train, k, 32, True),
                      epochs=hyperM.epochs,
                      steps_per_epoch=z_train.shape[0],
                      callbacks=[early_stopping_callback, checkpoint_callback],
                      verbose=0)
    train_end = time.time()
    train_time = train_end - train_strat 

    y_test_one_hot = to_categorical(y_test, num_classes=k) 
    # set verbose to 0 to silent the output
    test_loss, test_acc = gnn.model.evaluate(z_test,  y_test_one_hot, verbose=0) 
    return test_acc, train_time
############------------Sparse_supervised_learning_end---------###############


#Packages for Drive Files

In [7]:
# import packages
## for mount drive purpose
import os
from google.colab import drive

#Mount Drive

In [8]:
# mount drive
drive.mount('/content/drive/', force_remount=True)
os.chdir('/content/drive/My Drive/Colab_Notebooks/Graph_ML/semi_dr.shen')

Mounted at /content/drive/


# import ipynb packages

In [9]:
!pip install import-ipynb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 4.9 MB/s 


In [10]:
import import_ipynb
from test_cases import Model, Case

importing Jupyter notebook from test_cases.ipynb
Mounted at /content/drive/


In [ ]:
import import_ipynb
from test_cases_numba import Model, Case

importing Jupyter notebook from test_cases_numba.ipynb


# Test Cases

In [11]:
############------------case_data_preprocess_start------------------############
def adj_to_edg(A):
  """
    input is the symmetric adjacency matrix: A
    other variables in this function:
    s: number of edges
    return edg_list -- matrix format with shape(edg_sum,3):
    example row in edg_list(matrix): [vertex1, vertex2, connection weight from Adj matrix]
  """
  # check the len of the second dimenson of A
  if A.shape[1] <= 3:
    edg = A
  else:
    n = A.shape[0]
    # construct the initial edgg_list matrix with the size of (edg_sum, 3)
    edg_list = []
    for i in range(n):
      for j in range(i, n):
        if A[i,j] > 0:
          row = [i, j, A[i,j]]
          edg_list.append(row)
    edg = np.array(edg_list)
  return edg
############------------case_data_preprocess_end--------------------############

### Case_original vs case_numba

In [ ]:
n = 5000
case = Case(n)

#### Original-case

In [ ]:
start = time.time()
case_test = case.case_10_fully_known()
end = time.time()
print(end-start)

89.68082427978516


#### Numba-case

In [ ]:
start = time.time()
case_test = case.case_10_fully_known()
end = time.time()
print(end-start)

62.95595097541809


### GEE_Edge_list_Original vs GEE_Edge_list_Numba

#### Case 10 with 3000 nodes (SBM)

In [12]:
n = 3000
case = Case(n)

In [13]:
case_test = case.case_10_fully_known()
case_test.summary()

Info:

    SBM with 3 classes and defined probabilities with fully known labels
    80% for training and 20% for testing
    
n:
<class 'int'>
3000
d:
<class 'int'>
3
X:
(3000, 3000)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 1 0]]
Y:
(3000, 1)
[[1]
 [0]
 [2]
 ...
 [2]
 [2]
 [2]]


In [17]:
edge_list = adj_to_edg(case_test.X)

#### Laplacian = True, DiagA = False, Correlation = False -- original

In [18]:
GEE = GraphEncoderEmbed()
Z, W, emb_time = GEE.run(edge_list, case_test.Y, case_test.n, Laplacian = True, DiagA = False, Correlation = False)
print("--- embed %s seconds ---" % emb_time)

--- embed 7.517747640609741 seconds ---


#### Laplacian = True, DiagA = False, Correlation = False -- Numba

In [15]:
Z, W, emb_time = GEE_run(case_test.Y, case_test.n, edge_list, Laplacian = True, DiagA = False, Correlation = False)
print("--- embed %s seconds ---" % emb_time)

--- embed 7.582218408584595 seconds ---


#### Laplacian = True, DiagA = True, Correlation = True -- original

In [16]:
GEE = GraphEncoderEmbed()
Z, W, emb_time = GEE.run(edge_list, case_test.Y, case_test.n, Laplacian = True, DiagA = True, Correlation = True)
print("--- embed %s seconds ---" % emb_time)

--- embed 12.64570951461792 seconds ---


#### Laplacian = True, DiagA = True, Correlation = True -- Numba

In [18]:
Z, W, emb_time = GEE_run(case_test.Y, case_test.n, edge_list, Laplacian = True, DiagA = True, Correlation = True)
print("--- embed %s seconds ---" % emb_time)

--- embed 7.053375482559204 seconds ---


### Case 10 with 5000 nodes (SBM)

In [ ]:
n = 5000
case = Case(n)

In [ ]:
case_test = case.case_10_fully_known()
case_test.summary()

Info:

    SBM with 3 classes and defined probabilities with fully known labels
    80% for training and 20% for testing
    
n:
<class 'int'>
5000
d:
<class 'int'>
3
X:
(5000, 5000)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 0 0 0]]
Y:
(5000, 1)
[[1]
 [0]
 [2]
 ...
 [2]
 [0]
 [1]]


In [ ]:
edge_list = adj_to_edg(case_test.X)

#### Laplacian = True, DiagA = True, Correlation = True

##### Numba

In [ ]:
Z, W, emb_time = GEE_run(case_test.Y, case_test.n, edge_list, Laplacian = True, DiagA = True, Correlation = True)
print("--- embed %s seconds ---" % emb_time)

--- embed 16.475561141967773 seconds ---


##### Original

In [ ]:
GEE = GraphEncoderEmbed()
Z, W, emb_time = GEE.run(edge_list, case_test.Y, case_test.n, Laplacian = True, DiagA = True, Correlation = True)
print("--- embed %s seconds ---" % emb_time)

--- embed 19.47906732559204 seconds ---
